In [2]:
using Pkg
using Revise
Pkg.activate("..")
using JuliaClusterAndTreeTools
using Statistics
using Pkg
using RCall
using DataFrames
using StatsBase
using Distances
using CSV
using BioSymbols
using Clustering
R"""
library(ggplot2)
library(dplyr)
library(data.table)

"""



  Activating project at `/mnt/raid/covid-phylogeny/scripts/julia_modules/JuliaClusterAndTreeTools`


RObject{StrSxp}
 [1] "data.table" "dplyr"      "ggplot2"    "stats"      "graphics"  
 [6] "grDevices"  "utils"      "datasets"   "methods"    "base"      


In [3]:
swarm_clusters = "/mnt/raid/covid-phylogeny/audines11/rez/lineages/B.1.1.7/pair_id_Gap2a_derep1_swarm.fasta.swarminfo.cleaned.txt"
fasta = "/mnt/raid/covid-phylogeny/audines11/rez/lineages/B.1.1.7/pair_id_Gap2a_derep1.fasta"
cleaned_output = "/mnt/raid/covid-phylogeny/audines11/rez/lineages/B.1.1.7/pair_id_Gap2a_derep1_swarm.fasta.swarminfo.cleaned_single.txt"

"/mnt/raid/covid-phylogeny/audines11/rez/lineages/B.1.1.7/pair_id_Gap2a_derep1_swarm.fasta.swarminfo.cleaned_single.txt"

In [ ]:
function hclust_aln(m::Matrix{DNA},cut_distance=3,verbose=true)
    D = pairwise(hamming,m, dims=1)
    tr = hclust(D,:complete)
    cut = cutree(tr,h=cut_distance)
    return(cut)
end
    

In [6]:
aln = JuliaClusterAndTreeTools.Alignment(fasta)

Reading aligment /mnt/raid/covid-phylogeny/audines11/rez/lineages/B.1.1.7/pair_id_Gap2a_derep1.fasta



Alignment(["Denmark/DCGC-72547/2021", "USA/UT-UPHL-210811214020/2021", "Canada/Qc-L00331960001/2021", "Denmark/DCGC-72146/2021", "Denmark/DCGC-72492/2021", "Japan/PG-34525/2021", "Denmark/DCGC-72539/2021", "Denmark/DCGC-73912/2021", "Lithuania/NMVRI76159/2021", "Denmark/DCGC-74028/2021"  …  "Wales/PHWC-PYYRU5/2021", "Wales/QEUH-131C636/2021", "Wales/QEUH-13278BA/2021", "Wales/RAND-12D6F89/2021", "Zambia/SP32/2021", "env/Austria/CeMM5659/2021", "env/Austria/CeMM5673/2021", "env/Austria/CeMM6146/2021", "otter/USA/GA-21-011485-007/2021", "romania/SV_SJU_36296/2021"], DNA[DNA_G DNA_G … DNA_C DNA_T; DNA_G DNA_G … DNA_C DNA_T; … ; DNA_G DNA_G … DNA_C DNA_T; DNA_G DNA_G … DNA_C DNA_T], Int32[98597, 21858, 7221, 5395, 4822, 4747, 2917, 2875, 2741, 2575  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Dict{String, Int32}("England/QEUH-1345CAB/2021" => 32015, "France/ARA-HCL021063203301/2021" => 34246, "Netherlands/ZE-RIVM-26468/2021" => 44861, "Germany/SN-RKI-I-143744/2021" => 38403, "England/ALDP-133A113/20

In [9]:
clusters = DataFrame(CSV.File(swarm_clusters))
clusters_grouped = groupby(clusters, :Cluster)

Row,Member,Cluster
,String,String
1,Argentina/PAIS-A0549/2021,Argentina/PAIS-A0549/2021
2,USA/FL-CDC-STM-000053295/2021,Argentina/PAIS-A0549/2021
3,USA/NJ-CDC-ASC210026528/2021,Argentina/PAIS-A0549/2021
Row,Member,Cluster
,String,String
1,Wales/PHWC-PYYE99/2021,Wales/PHWC-PYYE99/2021
2,Wales/PHWC-4DE0CB/2021,Wales/PHWC-PYYE99/2021


In [19]:
clusters_grouped_big = filter(x -> length(x.Member) > 1000, clusters_grouped)[1:2]

Row,Member,Cluster
,String,String
1,Canada/Qc-L00331960001/2021,Canada/Qc-L00331960001/2021
2,USA/FL-CDC-STM-000056449/2021,Canada/Qc-L00331960001/2021
3,USA/AZ-CDC-STM-000057461/2021,Canada/Qc-L00331960001/2021
4,USA/MN-MDH-6345/2021,Canada/Qc-L00331960001/2021
5,USA/NY-P11E7/2021,Canada/Qc-L00331960001/2021
6,USA/PA-CDC-QDX24042180/2021,Canada/Qc-L00331960001/2021
7,USA/SC-DHEC-1592/2021,Canada/Qc-L00331960001/2021
8,USA/NY-Wadsworth-21046273-01/2021,Canada/Qc-L00331960001/2021
9,USA/KY-KSPHL-100525/2021,Canada/Qc-L00331960001/2021


In [17]:
size(clusters_grouped_big)

(2289, 2)

In [21]:
ct =0 
for group in collect(SubDataFrame, clusters_grouped_big)
    ct += 1
    names = group.Member
    aln_cluster = JuliaClusterAndTreeTools.sub_alignment()
    if ct > 0
        break
    end
end

Canada/Qc-L00331960001/2021


In [ ]:

names = collect(clusters_grouped_big[1].Member)
print(aln.name_sequence_map[names[1]])
print("")


n = length(names)
D = Array{Int64,2}(undef,n,n)
ct = 0
print(ct)
for i in 1:n
    for j in  i:n
        ct += 1
        idxi = aln.name_sequence_map[names[i]]
        idxj = aln.name_sequence_map[names[j]]
        si = aln.M[idxi,:]
        sj = aln.M[idxj,:]
        d = hamming(si,sj)
        D[i,j]=d
        
    end
    println("Done ", ct/(n*n/2),"\r")
end
tr = hclust(D,:complete,:U)
cut = cutree(tr,h=3)
print(cut)
